In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [2]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [3]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [4]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
0,2020-01-02 18:00:00,1.31883,1.31883,1.31225,1.31260,1.31260,1.31883,5.319149,100.000000,94.680851,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-02 19:00:00,1.31260,1.31336,1.31144,1.31332,1.31260,1.31332,60.416667,97.916667,37.500000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-02 20:00:00,1.31333,1.31427,1.31240,1.31386,1.31333,1.31386,49.732620,78.074866,28.342246,...,94.680851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-02 21:00:00,1.31385,1.31395,1.31270,1.31280,1.31280,1.31385,8.000000,92.000000,84.000000,...,37.500000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2020-01-02 22:00:00,1.31279,1.31446,1.31277,1.31376,1.31279,1.31376,1.183432,58.579882,57.396450,...,28.342246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_an = RSI(df_an)
df_an = BollingerBands(df_an)
df_an = ATR(df_an)
df_an = KeltnerChannels(df_an)
df_an = MACD(df_an)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,BB_MA,BB_UP,BB_LW,ATR_14,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST
0,2020-01-02 18:00:00,1.31883,1.31883,1.31225,1.31260,1.31260,1.31883,5.319149,100.000000,94.680851,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02 19:00:00,1.31260,1.31336,1.31144,1.31332,1.31260,1.31332,60.416667,97.916667,37.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-02 20:00:00,1.31333,1.31427,1.31240,1.31386,1.31333,1.31386,49.732620,78.074866,28.342246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-02 21:00:00,1.31385,1.31395,1.31270,1.31280,1.31280,1.31385,8.000000,92.000000,84.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-02 22:00:00,1.31279,1.31446,1.31277,1.31376,1.31279,1.31376,1.183432,58.579882,57.396450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,2023-03-23 11:00:00,1.23206,1.23319,1.23088,1.23153,1.23153,1.23206,28.138528,51.082251,22.943723,...,1.228631,1.235048,1.222214,0.001840,1.229357,1.233365,1.225349,0.001921,0.001728,0.000193
19995,2023-03-23 12:00:00,1.23154,1.23162,1.22841,1.22886,1.22886,1.23154,14.018692,97.507788,83.489097,...,1.228932,1.234908,1.222956,0.001961,1.229310,1.233654,1.224966,0.001643,0.001711,-0.000067
19996,2023-03-23 13:00:00,1.22886,1.23070,1.22822,1.23026,1.22886,1.23026,25.806452,82.258065,56.451613,...,1.229246,1.234641,1.223851,0.001979,1.229400,1.234026,1.224774,0.001519,0.001672,-0.000154
19997,2023-03-23 14:00:00,1.23026,1.23375,1.22650,1.22752,1.22752,1.23026,14.068966,51.862069,37.793103,...,1.229525,1.234243,1.224806,0.002462,1.229221,1.234603,1.223839,0.001185,0.001575,-0.000390


In [6]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST'],
      dtype='object')

In [7]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

9e-05

In [8]:
df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,KeLo,MACD,SIGNAL_MACD,HIST,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next
0,2020-01-06 03:00:00,1.30789,1.30806,1.30743,1.30783,1.30783,1.30789,63.492063,73.015873,9.523810,...,1.305453,-0.000768,-0.000886,0.000118,True,False,1.30759,1.30871,1.30778,1.30874
1,2020-01-06 04:00:00,1.30778,1.30874,1.30778,1.30863,1.30778,1.30863,0.000000,88.541667,88.541667,...,1.305891,-0.000647,-0.000832,0.000185,False,False,1.30743,1.30806,1.30798,1.30894
2,2020-01-06 05:00:00,1.30861,1.30894,1.30798,1.30810,1.30810,1.30861,12.500000,65.625000,53.125000,...,1.305948,-0.000586,-0.000778,0.000193,False,True,1.30778,1.30874,1.30798,1.30859
3,2020-01-06 06:00:00,1.30808,1.30859,1.30798,1.30831,1.30808,1.30831,16.393443,54.098361,37.704918,...,1.306168,-0.000514,-0.000722,0.000207,False,False,1.30798,1.30894,1.30788,1.30850
4,2020-01-06 07:00:00,1.30830,1.30850,1.30788,1.30799,1.30799,1.30830,17.741935,67.741935,50.000000,...,1.306212,-0.000477,-0.000670,0.000193,False,False,1.30798,1.30859,1.30676,1.30824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19957,2023-03-23 10:00:00,1.23105,1.23286,1.23045,1.23206,1.23105,1.23206,24.896266,66.804979,41.908714,...,1.225284,0.001976,0.001679,0.000297,True,False,1.23048,1.23434,1.23088,1.23319
19958,2023-03-23 11:00:00,1.23206,1.23319,1.23088,1.23153,1.23153,1.23206,28.138528,51.082251,22.943723,...,1.225349,0.001921,0.001728,0.000193,False,True,1.23045,1.23286,1.22841,1.23162
19959,2023-03-23 12:00:00,1.23154,1.23162,1.22841,1.22886,1.22886,1.23154,14.018692,97.507788,83.489097,...,1.224966,0.001643,0.001711,-0.000067,False,False,1.23088,1.23319,1.22822,1.23070
19960,2023-03-23 13:00:00,1.22886,1.23070,1.22822,1.23026,1.22886,1.23026,25.806452,82.258065,56.451613,...,1.224774,0.001519,0.001672,-0.000154,False,False,1.22841,1.23162,1.22650,1.23375


In [9]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0

for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] #- SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i] #+ SPREAD
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] #+ SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1003418580.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = SELL
C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1003418580.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.high[i-1] #+ SPREAD
C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1003418580.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_an.close

In [10]:
df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

In [11]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST', 'pivots_l', 'pivots_h', 'low_perv', 'high_perv',
       'low_next', 'high_next', 'SIGNAL', 'TP', 'SL'],
      dtype='object')

In [86]:
X = []
Y = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces = []
for i in range(4,len(df_an)):
#     if df_an.SIGNAL[i] != NONE:
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        feature.append(df_an.pivots_h[i-2])
        feature.append(df_an.pivots_l[i-2])
        feature.append(df_an.pivots_h[i-3])
        feature.append(df_an.pivots_l[i-3])
        feature.append(df_an.pivots_h[i-4])
        feature.append(df_an.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [87]:
print(len(X))
print(len(Y))
print(len(indeces))

8644
8644
8644


In [88]:
print(len(feature))

208


In [89]:
SPLIT_INDEX = int(0.8*len(X))

In [90]:

X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]


In [91]:
print(len(X_train))
print(len(Y_train))
print(len(indeces_train))
print(len(X_test))
print(len(Y_test))
print(len(indeces_test))

6915
6915
6915
1729
1729
1729


In [92]:
# scaler = StandardScaler()  
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test) 


In [93]:
# help(MLPClassifier)

In [94]:
clf = MLPClassifier( solver='adam', alpha=1e-5,learning_rate_init=1e-4,
                     tol=0.00000010,
                    hidden_layer_sizes=(200,200), 
                    random_state=1,max_iter=100000,verbose=True,warm_start = True)
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [95]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 2.81823984
Iteration 2, loss = 1.60685825
Iteration 3, loss = 1.23607110
Iteration 4, loss = 1.09500381
Iteration 5, loss = 1.02468854
Iteration 6, loss = 0.97983142
Iteration 7, loss = 0.95175996
Iteration 8, loss = 0.92026072
Iteration 9, loss = 0.89362001
Iteration 10, loss = 0.87628728
Iteration 11, loss = 0.87519531
Iteration 12, loss = 0.85949850
Iteration 13, loss = 0.85703907
Iteration 14, loss = 0.84827998
Iteration 15, loss = 0.82805833
Iteration 16, loss = 0.81213849
Iteration 17, loss = 0.80454821
Iteration 18, loss = 0.79782524
Iteration 19, loss = 0.80601206
Iteration 20, loss = 0.81310888
Iteration 21, loss = 0.78095543
Iteration 22, loss = 0.77794464
Iteration 23, loss = 0.76979371
Iteration 24, loss = 0.76671773
Iteration 25, loss = 0.77054499
Iteration 26, loss = 0.75793542
Iteration 27, loss = 0.75671381
Iteration 28, loss = 0.74961784
Iteration 29, loss = 0.75327875
Iteration 30, loss = 0.74572353
Iteration 31, loss = 0.75583170
Iteration 32, los

Iteration 253, loss = 0.33434673
Iteration 254, loss = 0.32576850
Iteration 255, loss = 0.32625982
Iteration 256, loss = 0.33428712
Iteration 257, loss = 0.32793236
Iteration 258, loss = 0.32882939
Iteration 259, loss = 0.31906245
Iteration 260, loss = 0.32527378
Iteration 261, loss = 0.33844237
Iteration 262, loss = 0.34415995
Iteration 263, loss = 0.32201556
Iteration 264, loss = 0.31727778
Iteration 265, loss = 0.32198360
Iteration 266, loss = 0.31282091
Iteration 267, loss = 0.31219966
Iteration 268, loss = 0.31146137
Iteration 269, loss = 0.31437417
Iteration 270, loss = 0.31492781
Iteration 271, loss = 0.31158951
Iteration 272, loss = 0.30677426
Iteration 273, loss = 0.30523899
Iteration 274, loss = 0.31214175
Iteration 275, loss = 0.30908725
Iteration 276, loss = 0.30196317
Iteration 277, loss = 0.31175189
Iteration 278, loss = 0.30112538
Iteration 279, loss = 0.30286003
Iteration 280, loss = 0.30723441
Iteration 281, loss = 0.30578265
Iteration 282, loss = 0.29571649
Iteration 

Iteration 502, loss = 0.12981774
Iteration 503, loss = 0.12478021
Iteration 504, loss = 0.12595128
Iteration 505, loss = 0.12596103
Iteration 506, loss = 0.12611814
Iteration 507, loss = 0.12273806
Iteration 508, loss = 0.12124893
Iteration 509, loss = 0.12602012
Iteration 510, loss = 0.12077756
Iteration 511, loss = 0.12059760
Iteration 512, loss = 0.11815795
Iteration 513, loss = 0.12026037
Iteration 514, loss = 0.11863373
Iteration 515, loss = 0.11762214
Iteration 516, loss = 0.11549899
Iteration 517, loss = 0.11507188
Iteration 518, loss = 0.11374947
Iteration 519, loss = 0.12169364
Iteration 520, loss = 0.11460063
Iteration 521, loss = 0.11664189
Iteration 522, loss = 0.11591479
Iteration 523, loss = 0.11513578
Iteration 524, loss = 0.11311039
Iteration 525, loss = 0.11005288
Iteration 526, loss = 0.11307398
Iteration 527, loss = 0.11384323
Iteration 528, loss = 0.11012099
Iteration 529, loss = 0.11221071
Iteration 530, loss = 0.11097017
Iteration 531, loss = 0.11266181
Iteration 

Iteration 751, loss = 0.04230723
Iteration 752, loss = 0.03158973
Iteration 753, loss = 0.02988542
Iteration 754, loss = 0.03415609
Iteration 755, loss = 0.03101607
Iteration 756, loss = 0.02941691
Iteration 757, loss = 0.03297281
Iteration 758, loss = 0.03192926
Iteration 759, loss = 0.03083954
Iteration 760, loss = 0.03027798
Iteration 761, loss = 0.03264670
Iteration 762, loss = 0.02989958
Iteration 763, loss = 0.03619659
Iteration 764, loss = 0.03052903
Iteration 765, loss = 0.02819823
Iteration 766, loss = 0.02810268
Iteration 767, loss = 0.02763949
Iteration 768, loss = 0.02941543
Iteration 769, loss = 0.04032553
Iteration 770, loss = 0.03316316
Iteration 771, loss = 0.02801954
Iteration 772, loss = 0.02749586
Iteration 773, loss = 0.02672318
Iteration 774, loss = 0.02854878
Iteration 775, loss = 0.02747052
Iteration 776, loss = 0.02970780
Iteration 777, loss = 0.02656887
Iteration 778, loss = 0.02607608
Iteration 779, loss = 0.02889371
Iteration 780, loss = 0.02578341
Iteration 

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(200, 200),
              learning_rate_init=0.0001, max_iter=100000, random_state=1,
              tol=1e-07, verbose=True, warm_start=True)

In [96]:
y_perdict = clf.predict(X_test)

In [97]:
confusion_matrix(Y_test, y_perdict)

array([[138, 204,   7],
       [197, 669, 183],
       [  7, 186, 138]], dtype=int64)

In [98]:
# pickle.dump(clf, open(f"{pair}.pickle", "wb"))

In [99]:
(Y_test==y_perdict).sum()/len(y_perdict)

0.5465587044534413

In [72]:
print(len(indeces_test))
print(len(y_perdict))
# y_perdict

1729
1729


In [73]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST', 'pivots_l', 'pivots_h', 'low_perv', 'high_perv',
       'low_next', 'high_next', 'SIGNAL', 'TP', 'SL'],
      dtype='object')

In [74]:
indeces_test[0]

16001

In [100]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
        if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\4200224076.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [101]:
df_test

,time,open,high,low,close,TP,SL,pivots_l,pivots_h,SIGNAL
0,2022-08-02 07:00:00,1.22567,1.22619,1.22432,1.22479,1.22165,1.22636,0.0,0.0,0
1,2022-08-02 08:00:00,1.22479,1.22551,1.22384,1.22435,0.00000,0.00000,0.0,0.0,0
2,2022-08-02 09:00:00,1.22435,1.22435,1.22181,1.22198,0.00000,0.00000,0.0,0.0,0
3,2022-08-02 10:00:00,1.22198,1.22218,1.21884,1.22006,0.00000,0.00000,0.0,0.0,0
4,2022-08-02 11:00:00,1.22006,1.22054,1.21849,1.21906,0.00000,0.00000,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
3961,2023-03-23 10:00:00,1.23105,1.23286,1.23045,1.23206,1.22750,1.23434,1.0,0.0,0
3962,2023-03-23 11:00:00,1.23206,1.23319,1.23088,1.23153,0.00000,0.00000,0.0,1.0,1
3963,2023-03-23 12:00:00,1.23154,1.23162,1.22841,1.22886,0.00000,0.00000,0.0,0.0,0
3964,2023-03-23 13:00:00,1.22886,1.23070,1.22822,1.23026,0.00000,0.00000,0.0,0.0,0


In [102]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51558 entries, 0 to 51557
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    51558 non-null  datetime64[ns]
 1   open    51558 non-null  float64       
 2   high    51558 non-null  float64       
 3   low     51558 non-null  float64       
 4   close   51558 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 2.0 MB


In [103]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [104]:
df_res_m5 = gt.run_test()

run_test...
Result:
win rate:  0.6257763975155279
 2.0    403
-1.0    241
Name: result, dtype: int64


In [105]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1186191000.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1186191000.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [106]:
min(df_res_m5['balance'])

87.89519896170103

In [107]:
df_res_m5.result.value_counts()

 2.0    403
-1.0    241
Name: result, dtype: int64

In [110]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])

In [109]:
#  cross test
pair_cross = "EURUSD"
NB_H1_CANDLES_CROSS = 2000
rates = Rates(pair_cross, NB_H1_CANDLES_CROSS, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an_cross = rates.get_rates_from_now()
df_an_cross.drop(NB_H1_CANDLES_CROSS-1,inplace=True) 
df_an_cross.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an_cross = apply_patterns(df_an_cross)
for cp in candle_patterns:
    df_an_cross[cp] = df_an_cross[cp].astype(float)

df_an_cross = RSI(df_an_cross)
df_an_cross = BollingerBands(df_an_cross)
df_an_cross = ATR(df_an_cross)
df_an_cross = KeltnerChannels(df_an_cross)
df_an_cross = MACD(df_an_cross)

df_an_cross['pivots_l'] = False
df_an_cross['pivots_h'] = False

df_an_cross['low_perv'] = df_an_cross.low.shift(1)
df_an_cross['high_perv'] = df_an_cross.high.shift(1)
df_an_cross['low_next'] = df_an_cross.low.shift(-1)
df_an_cross['high_next'] = df_an_cross.high.shift(-1)
df_an_cross['pivots_l'] = (df_an_cross.low_perv > df_an_cross.low) & (df_an_cross.low_next > df_an_cross.low)
df_an_cross['pivots_h'] = (df_an_cross.high_perv < df_an_cross.high) & (df_an_cross.high_next < df_an_cross.high)
df_an_cross.dropna(inplace=True)
df_an_cross.reset_index(drop=True, inplace=True)

df_an_cross.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,KeLo,MACD,SIGNAL_MACD,HIST,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next
0,2022-11-25 18:00:00,1.04067,1.04114,1.03917,1.04014,1.04014,1.04067,49.238579,76.142132,26.903553,...,1.034850,-0.000448,-0.000404,-0.000044,False,True,1.03764,1.04070,1.03990,1.04094
1,2022-11-25 19:00:00,1.04013,1.04094,1.03990,1.04059,1.04013,1.04059,22.115385,66.346154,44.230769,...,1.034963,-0.000356,-0.000393,0.000037,False,False,1.03917,1.04114,1.03969,1.04061
2,2022-11-25 20:00:00,1.04060,1.04061,1.03969,1.04016,1.04016,1.04060,51.086957,98.913043,47.826087,...,1.035182,-0.000313,-0.000376,0.000063,True,False,1.03990,1.04094,1.03988,1.04150
3,2022-11-25 21:00:00,1.04016,1.04150,1.03988,1.04118,1.04016,1.04118,17.283951,80.246914,62.962963,...,1.035876,-0.000198,-0.000338,0.000139,False,True,1.03969,1.04061,1.04077,1.04142
4,2022-11-25 22:00:00,1.04118,1.04142,1.04077,1.04087,1.04087,1.04118,15.384615,63.076923,47.692308,...,1.036413,-0.000130,-0.000294,0.000164,False,False,1.03988,1.04150,1.03847,1.04103


In [39]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

4e-05

In [40]:
BUY = 1
SELL = -1
NONE = 0

df_an_cross['SIGNAL'] = 0
df_an_cross['TP'] = 0
df_an_cross['SL'] = 0

for i in range(1,len(df_an_cross)):
    if df_an_cross.pivots_l[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.low[j] <= df_an_cross.low[i-1] ):
                break
            else:
                if(df_an_cross.high[j] > 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = BUY
                    df_an_cross['SL'][i] = df_an_cross.low[i-1] #- SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i] #+ SPREAD
                    break
    if df_an_cross.pivots_h[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.high[j] >= df_an_cross.high[i-1] ):
                break
            else:
                if(df_an_cross.low[j] < 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = SELL
                    df_an_cross['SL'][i] = df_an_cross.high[i-1] #+ SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\3328372866.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\3328372866.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
df_an_cross.pivots_l = df_an_cross.pivots_l.astype(float)
df_an_cross.pivots_h = df_an_cross.pivots_h.astype(float)

In [42]:
X_cross = []
Y_cross = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces_cross = []
for i in range(4,len(df_an_cross)):
    if df_an_cross.pivots_h[i-1] or df_an_cross.pivots_l[i-1]:
#         print("im here")
        feature = []
        feature.append(df_an_cross.pivots_h[i-1])
        feature.append(df_an_cross.pivots_l[i-1])
        for fc in feature_col:
            feature.append(df_an_cross[fc][i])
            feature.append(df_an_cross[fc][i-1])
            feature.append(df_an_cross[fc][i-2])
            feature.append(df_an_cross[fc][i-3])
            feature.append(df_an_cross[fc][i-4])

        X_cross.append(feature)
        Y_cross.append(df_an_cross.SIGNAL[i])
        indeces_cross.append(i)

In [43]:
len(feature)

202

In [44]:
# clf = pickle.load(open("GBPUSD.pickle", "rb"))

In [45]:
y_perdict_cross = clf.predict(X_cross)

In [46]:
confusion_matrix(Y_cross, y_perdict_cross)

array([[  0,   0,   0],
       [175, 477, 162],
       [  0,   0,   0]], dtype=int64)

In [47]:
(Y_cross==y_perdict_cross).sum()/len(y_perdict_cross)

0.585995085995086

In [48]:
df_test = df_an_cross[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']]#.iloc[indeces_cross[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_cross):
    if i > 0:
        if df_test.pivots_l[idx-1] and y_perdict_cross[i] == BUY:
            df_test.SIGNAL.loc[idx] = y_perdict_cross[i]
        if df_test.pivots_h[idx-1] and y_perdict_cross[i] == SELL:
#         if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict_cross[i]

df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\732658451.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\732658451.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\732658451.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
rates = Rates(pair_cross, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    25545 non-null  datetime64[ns]
 1   open    25545 non-null  float64       
 2   high    25545 non-null  float64       
 3   low     25545 non-null  float64       
 4   close   25545 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 998.0 KB


In [50]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [51]:
df_res_m5 = gt.run_test()

run_test...
Result:
win rate:  0.6392405063291139
 2.0    202
-1.0    114
Name: result, dtype: int64


In [52]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1186191000.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1186191000.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
df_res_m5_plot = df_res_m5#.loc[:200]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])